# Week 3
### Context

#### Ensemble
+ Voting Ensemble
+ Out-of-fold(OOF) Ensemble
+ Stacking Ensemble

In [ ]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

n_cpus = multiprocessing.cpu_count()

In [ ]:
BASE_DIR = '../'

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.info()

In [ ]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],         inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

#### 라벨 데이터 인코딩

In [ ]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    
    # 스케일링

    # 인코딩
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

## Ensemble
개인적으로 앙상블은 머신러닝의 꽃이라고 생각합니다. 단일 모델로 좋은 성능을 이끄는 것도 중요하지만, 서로 다른 모델의 다양성을 고려하여 결과를 이끌어내는 앙상블은 응용할 수 있는 방법이 매우 많습니다. <br>
그 중 대표적인 2가지 앙상블에 대해 실습하고 배워보도록 하겠습니다. 

### 1. Voting Ensemble
이름에서 알 수 있듯이 각자의 모델이 투표를 하여 클래스를 선택하는 방식의 앙상블 입니다. <br>
Voting 앙상블은 Sklearn 자체적으로 모델로써 지원을 하며, 사용하기도 매우 쉽습니다. <br>
그리고 Hard, Soft로 Voting 방식이 나뉘는데, Hard는 라벨 값으로 투표를 하는 방식이고, Soft는 확률 값을 모두 더해 가장 높은 클래스를 선택합니다.

Voting Classifier는 Sklearn의 ensemble 패키지에 있습니다.

In [ ]:
from sklearn.model_selection import train_test_split

# 쪼개어진 Train, Valid 데이터의 비율은 (7:3), 내부 난수 값 42, 데이터를 쪼갤 때 섞으며 label 값으로 Stratify 하는 코드 입니다. random_state를 주석 처리하고 데이터를 확인해보시면 계속 바뀝니다.
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True,
                                                      stratify=label)

In [ ]:
x_train, x_valid, _ = preprocess(x_train, x_valid, test)

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

clfs = [['Logistic', LogisticRegression()],
        ['RandomForest', RandomForestClassifier()],
        ['MLP', MLPClassifier()]]

vote_clf = VotingClassifier(clfs, voting='soft', n_jobs=4)

#### 2) 모델 학습하기

In [ ]:
# 여기서 x_train, y_train은 마지막 Fold
vote_clf.fit(x_train, y_train)

#### 3) 결과 확인하기

In [ ]:
from sklearn.metrics import f1_score, log_loss

print('Validation F1 score : {:.4f}'.format(f1_score(y_valid, vote_clf.predict(x_valid), average='weighted')))

In [ ]:
# voting hard 에서는 작동 x, 확률로 값을 뽑을 수 없음.
# print('Validation log_loss score : {:.4f}'.format(log_loss(y_valid, vote_clf.predict_proba(x_valid))))

### 2. Out-of-fold(OOF) Ensemble (같이 푸는 실습)
OOF 앙상블은 KFold 교차 검증에서 생성되는 각 Fold에 대한 예측 값을 앙상블하는 기법으로 모델 검증과 함께 앙상블을 진행할 수 있다는 장점이 있습니다. <br>

Cross Validation 파트에서 배웠던 KFold 코드를 재사용해 OOF 앙상블을 진행해보겠습니다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

val_scores = list()
oof_pred = np.zeros((#맞는 차원 집어넣기))
n_splits=5
    
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    
    
    # 모델 정의
    
    
    # 모델 학습

    
    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = 
    val_logloss = 
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    oof_pred += model.predict_proba(x_test) / skf.n_splits 

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

In [ ]:
submit_path = join(BASE_DIR, 'data', 'MDC14', 'sample_submission.csv')

submit = pd.read_csv(submit_path)
submit

In [ ]:
submit.iloc[:, 1:] = oof_pred
submit

In [ ]:
# submit.to_csv('oof_first_submit.csv', index=False)

## Two Stage Ensemble
### Stacking
- 지난 수업에 kFold를 활용한 OOF 앙상블에 대해 학습했습니다. Stacking은 말 그대로 모델의 결과를 쌓아서 앙상블을 하는 방식입니다.
- 스태킹의 원리는, 모델이 예측한 y 값은 "실제 y 값과 매우 선형성이 높다는 점을 이용하여 y_train_pred를 변수로 사용합니다.
- y_train_pred는 모든 vaild fold의 예측 값을 합쳐서 만듭니다.

1. 우선 서로 다른 모델에 대해 y_train_pred, y_test_pred 값을 모읍니다.
2. 모은 y_train_pred 값들을 axis=1 방향으로 합친 데이터를 new_x_train이라고 하겠습니다. 
    - 당연히 y_test_pred 값들도 axis=1 방향으로 합쳐 new_x_test라고 합니다.
3. new_x_train 데이터와, y_train 데이터로 모델을 학습하고, new_x_test로 최종 y_test_pred를 예측합니다.
    - 이때 주로 2 stage 모델(meta model이라고도 부릅니다.)은 성능이 강력한 모델을 사용합니다. (사실 해봐야 xgb, lgb 입니다..)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

val_scores = list()
# 결과 값들을 stacking 해야하기 때문에, (모델 개수, 샘플의 수, 3) 라는 차원으로 구성됩니다.
oof_train = np.zeros((6, data.shape[0], 3))
oof_pred  = np.zeros((6, test.shape[0], 3))

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    models = [RandomForestClassifier(n_estimators=200, max_depth=5,  random_state=42, n_jobs=(n_cpus-1)),
              RandomForestClassifier(n_estimators=150, max_depth=8, random_state=42, n_jobs=(n_cpus-1)),
              XGBClassifier(n_estimators=200, max_depth=5, random_state=42, n_jobs=(n_cpus-1)),
              XGBClassifier(n_estimators=150, max_depth=8, random_state=42, n_jobs=(n_cpus-1)),
              LGBMClassifier(n_estimators=200, max_depth=5, random_state=42, n_jobs=(n_cpus-1)),
              LGBMClassifier(n_estimators=150, max_depth=8, random_state=42, n_jobs=(n_cpus-1))]
            
    for j, model in enumerate(models):
        # 모델 학습
        model.fit(x_train, y_train)

        # j번째 칸에 맞는 결과 담기.
        oof_train[j, val_idx,] += model.predict_proba(x_valid)
        oof_pred[j, :,]        += model.predict_proba(x_test) / n_splits
    
    print(f'{i} Fold, ...')

In [ ]:
oof_train.T.shape

In [ ]:
# 모은 train, test의 예측 값을 new_x_train, new_x_test로 사용합니다.
new_train = pd.DataFrame(np.concatenate(oof_train.T, axis=1))
new_test  = pd.DataFrame(np.concatenate(oof_pred.T, axis=1))

In [ ]:
new_train.shape, new_test.shape

#### OOF 앙상블을 진행합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

val_scores = list()
oof_pred  = np.zeros((test.shape[0], 3))

for i, (trn_idx, val_idx) in enumerate(skf.split(new_train, label)):
    x_train, y_train = new_train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = new_train.iloc[val_idx, :], label[val_idx]
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train     = scaler.transform(x_train)
    x_valid     = scaler.transform(x_valid)
    new_x_test  = scaler.transform(new_test)

    # 모델 정의
    model = XGBClassifier(random_state=42, n_jobs=(n_cpus-1))
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    # 반드시 log의 역함수인 exp를 취해주세요.
    oof_pred += model.predict_proba(new_x_test) / n_splits
    

# 교차 검증 log loss 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

In [ ]:
submit.iloc[:, 1:] = oof_pred
submit

In [ ]:
# submit.to_csv('stacking_first_submit.csv', index=False)

## 실습 솔루션

### 전처리 함수 만들기

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### Out-of-fold ensemble 실습 

In [ ]:
from sklearn.model_selection import StratifiedKFold

val_scores = list()
oof_pred = np.zeros((test.shape[0], 3))
n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    oof_pred += model.predict_proba(x_test) / skf.n_splits 

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))